# HDS5210-2022 Fall Midterm

In the midterm exercise, you're going to use all the programming and data management skills you've developed so far to build a pricing calculator that will calculate how much money should be reimbursed for the visits in a CSV file. To do this, you will need to get allowed amounts (aka rates) from a JSON file, apply some special rules, and then calculate various totals by hospital or by month.

Each step of the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure you're getting each step correct.

All functions require docstrings with a description and at least one test case.

The midterm is due Monday, October 24th at 11:59 PM CST.

---

## Step 1: Average Rate

With the `/data/negotiated_rates.json` file as input for your first function, read all the `allowed_amount` attributes and calculate an average allowed amount over all rates in the file.

Your function should be named **average_rate()**, take the file's name as it's input parameter, and return a float as the result.

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [90]:
import json

def average_rate(file):
    """
    This method is used to get the average of all the allowed_amount attribute values present in the data file taking 
    the file as an input parameter
    """
    with open(file,'r') as f:
        file=json.load(f)
    values=[]
    for i in range(len(file['out_of_network'])):
        for j in range(len(file['out_of_network'][i]['allowed_amounts'])):
            values.append(file['out_of_network'][i]['allowed_amounts'][j]['payments']['allowed_amount'])
    return (round(sum(values)/len(values),2))

    

In [91]:
assert(average_rate('/data/negotiated_rates.json') == 38.67)

In [92]:
average_rate('/data/negotiated_rates.json')

38.67

---

## Step 2: Rate for a Billing Code / Service Code Combination

For the next step, we need to be able to look up the allowed amount for any given billing code / service code combination.

In this data, the billing code represents the service or procedure that was provided. The service code represents the type of site where the service was provided.

Your function should be named **get_rate()** and take three parameters: the JSON file name, the billing code, and the service code. If your code can't find that combination in the file, it should return None.

In [219]:
import json

def get_rate(file, billing_code, service_code):
    """
    This function takes the file, billing code and service code as parameters and returns the allowed_amount
    pertaining to the input parameters passed.
    """
    with open(file,'r') as f:
        file=json.load(f)
    value=None
    for i in range(len(file['out_of_network'])):
        if(file['out_of_network'][i]['billing_code']==billing_code):
            for j in range(len(file['out_of_network'][i]['allowed_amounts'])):
                if(file['out_of_network'][i]['allowed_amounts'][j]['service_code']==service_code):
                    value=file['out_of_network'][i]['allowed_amounts'][j]['payments']['allowed_amount']
    return value
    

    

In [220]:
assert (get_rate('/data/negotiated_rates.json','G0283','11') == 8.78)
assert (get_rate('/data/negotiated_rates.json','97140','11') == 20.03)
assert (get_rate('/data/negotiated_rates.json','97110','12') == 26.62)

---

## Part 3: Special Rules for Rates

There are some special rules for adjusting rates depending on a patient's age and the day of week.  (These are made up. Not from the real world.) In the next part of the midterm, you'll need to create a function that applies these extra rules to a rate and return the adjusted value.

1. If the day of week is Monday, charge only 75% of the allowed amount.
2. If the patient's age is 65 or higher, charge only 50% of the allowed amount.
3. If's both Monday and the patient's age is 65 or higher, charge only 50% of the allowed amount.
4. If neither of these conditions are true, charge the whole amount.

Your function should be named **get_adjusted_rate()** and take five parameters: file name, billing code, service code, patient age, and visit date.  Your function should return the adjusted rate (based on the rules above) or None if the rate couldn't be found in the file.

Note that your function will take a date in the form `%Y-%m-%d` ([see datetime.strptime()](https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime)) and will need to calculate the [day of week](https://docs.python.org/3/library/datetime.html#datetime.date.weekday).

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [221]:
from datetime import datetime
def get_adjusted_rate(file, billing_code, service_code, patient_age, visit_date):
    """
    This function is used to adjust the rates of patients based on the following conditions:
    1. If Week of day is Monday, we charge 75% 
    2. If patient's age is greater than 65, we charge 50%
    3. If its both Monday and age is greater than 65, we charge 50%
    4. If none of the above, we charge the full amount
    """
    visitDate = datetime.strptime(visit_date,"%Y-%m-%d")
    day = visitDate.strftime('%A')
    actual_rate = get_rate(file, billing_code, service_code)
    if actual_rate==None:
        return None
    try:
        if day == 'Monday' and patient_age >= 65:
            adjusted_rate = 0.5 *actual_rate
        elif day == 'Monday':
            adjusted_rate = 0.75 * actual_rate  
        elif patient_age >= 65:
            adjusted_rate = 0.5 * actual_rate
        else:
            return actual_rate
        return round( adjusted_rate, 2)
    except:
        return None
   


            
        

    
    

In [222]:
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07') == 8.78)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03') == 6.58)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-03') == 4.39)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-08') == 4.39)

In [223]:
print(get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07'))
print(get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03'))
print(get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-03'))
print(get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-08'))

8.78
6.58
4.39
4.39


---

## Step 4: Calculate total payments for a list of visits

Last step, we're going to put your get_adjusted_rate() function to work on a list of visits from a few hospitals in the St. Louis region.

Your last function, **summarize_reimbursement()** needs to read an encounter file and the negotiated rates file, compute an adjusted rate for each encounter (row) in the input file, and return two dictionaries of information:
1. Total expected reimbursement by month
2. Total expected reimbursement by hospital

Your should be able to run your function as `by_month, by_hospital = summarize_reimbursement(visits, negotiated_rates)` and have the two answers below:

**by_month**
```json
{
    '2021-05': 192.38,
    '2021-03': 378.72,
    '2021-07': 277.67,
    '2021-06': 236.53,
    '2021-11': 229.7,
    '2021-10': 234.52,
    '2021-12': 297.87,
    '2021-04': 337.7,
    '2021-09': 160.4,
    '2021-01': 111.91,
    '2021-02': 158.55,
    '2021-08': 152.28
}
```

**by_hospital**
```json
{
    'Missouri Baptist': 514.18,
    'SSM DePaul': 460.02,
    'SLU Hospital': 409.67,
    'Barnes Jewish': 485.7,
    'Mercy Springfield': 518.59,
    'Mercy St. Louis': 380.07
}
```

**Round your totals to 2 decimal places**

**If rates are not found, just ignore them**

In [224]:
import json 
import csv
from datetime import datetime
def summarize_reimbursement(visits, negotiated_rates):
    """
    This function returns the expected values of patient visits grouped by month and hospital
    """
    by_month = {}
    by_hospital = {}
    with open(visits, 'r') as cf:
        _csv = csv.DictReader(cf)
        for row in _csv:
            bi_c = row[ 'BILLING_CODE']
            se_c = row[ 'SERVICE_CODE']
            patient_age = row['PATIENT_AGE']
            v_d = row[ 'VISIT_DATE']
            hp = row[ 'HOSPITAL']
            date = datetime. strptime (v_d, "%Y-%m-%d")
            m = date.strftime ('%Y-%m') #month
            rt = get_adjusted_rate (
            negotiated_rates, bi_c, se_c, int(patient_age), v_d
            )
            if m in by_month:
                adj_rate = by_month[m]
                if rt is None:
                    rt = 0
                    by_month[m] = round(adj_rate + rt, 2)
                else:
                    by_month[m] = round(adj_rate + rt, 2)     
            else:
                if rt is not None:
                    by_month[m] = round(rt, 2)
                else:
                    by_month[m] = 0
            if hp in by_hospital:
                adj_rate - by_hospital[hp]
                if rt is not None:
                    by_hospital[hp] = round(adj_rate + rt, 2)
                else:
                    rt = 0
                    by_hospital[hp] = round(adj_rate + rt, 2)
            else:
                    if rt is not None:
                        by_hospital[row['HOSPITAL']] = round(rt, 2)
                    else:
                        by_hospital[row['HOSPITAL']] = 0
    return by_month, by_hospital


            

In [225]:
summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')

({'2021-05': 192.38,
  '2021-03': 378.72,
  '2021-07': 277.67,
  '2021-06': 236.53,
  '2021-11': 229.7,
  '2021-10': 234.52,
  '2021-12': 297.87,
  '2021-04': 337.7,
  '2021-09': 160.4,
  '2021-02': 158.55,
  '2021-01': 111.91,
  '2021-08': 152.28},
 {'Missouri Baptist': 127.64,
  'SSM DePaul': 236.53,
  'SLU Hospital': 111.91,
  'Mercy St. Louis': 234.52,
  'Barnes Jewish': 152.28,
  'Mercy Springfield': 87.27})

In [227]:
print(summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07'])
print(summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-03'])

277.67
378.72


In [226]:
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07'] == 277.67)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-03'] == 378.72)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy St. Louis'] == 380.07)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy Springfield'] == 518.59)

AssertionError: 